# 多クラスロジスティック回帰

## softmaxを使う理由

二値分類のロジスティック回帰では、1つの線形回帰モデルの出力を確率値にしたかったため、sigmoid関数で0~1にスケーリングすれば良かった。一方、kクラスロジスティック回帰は、k 個の独立した線形回帰モデルを用いつつ、その出力値同士を比較して相対的に値が大きなクラスの確率が高い状態で最終出力を出す関数を、活性化関数として利用したい。

この活性化関数が担うべき役割は下記の通り。
- k 個の値を入力に k 個の値を出力する
  - k クラスそれぞれの確率を出力したいため
- 出力する k 個の総和は 1 になっている
  - 確率が合わせて1を超えるとおかしいため
- 出力する k 個の値は全て 0 ~ 1 の間の値である
  - 確率値がマイナスや1以上だとおかしいため


softmax関数はこの要件を満たすため、多クラスロジスティック回帰では活性化関数にsoftmax関数を採用する。

## 多クラス分類問題（カテゴリカル分布）における対数尤度の導出

${\bf x^{(n)}}$ がクラス $i$ かどうかを表す変数 $y^{(n)}_i \in {0, 1}$を考える。ただし、$\sum_i^K{y^{(n)}_i} = 1$である。（${\bf x^{(n)}}$は必ず一つのクラスに属するとする） 

すると、あるデータ点 ${\bf x}^{(n)}$ がクラス $i$ である確率は、下記のような事後確率で表現することができる

$$
p(y^{(n)}_0 = 1 | {\bf x}) = \tilde{Y}_0({\bf x}^{(n)};{\bf w}) \\
p(y^{(n)}_1 = 1 | {\bf x}) = \tilde{Y}_1({\bf x}^{(n)};{\bf w}) \\
\vdots \\
p(y^{(n)}_k = 1 | {\bf x}) = \tilde{Y}_k({\bf x}^{(n)};{\bf w}) \\
$$

ここで、正解クラスのビットだけが1, それ以外0の 1-hot ベクトル ${\bf y}^{(n)}$ を導入すると、下記のような一つの式でまとめて表すことができる。

$$
p({\bf y}^{(n)}|{\bf x}^{(n)}) = \prod^k_{i=1}{ \tilde{Y}_i({\bf x}^{(n)};{\bf w}) ^{y^{(n)}_i} }
$$

尤度は上式で表されるような確率密度の積で表すことができたので、

$$
P = \prod^N_{n=1} { p({\bf y}^{(n)}|{\bf x}^{(n)}) } = \prod^N_{n=1} { \prod^k_{i=1} { \{\tilde{Y_i}({\bf x}^{(n)};{\bf w})\}^{y^{(n)}_i} } }
$$

対数尤度を取ると、

$$
\ln P = \sum^N_{n=1}\sum^k_{i=1} y_i^{(n)} \ln \tilde{Y_i}({\bf x}^{(n)};{\bf w}) \tag{1}
$$

## 損失関数から負の対数尤度（交差エントロピー）へ

二値分類のロジスティック回帰では、対数尤度を最大化することを考えたが、最大にすべき値を探すよりも、最小にすべき値を探す方が、一般的な最適化問題と形が合う。そのため、負の対数尤度を最小化することを考える。

※二値分類の方でも同じように負の対数尤度を最小化することもできる

負の対数尤度は、式(1) の負を取っただけなので、以下の通り。

$$
-\ln P = -\sum^N_{n=1}\sum^k_{i=1} y_i^{(n)} \ln \tilde{Y_i}({\bf x}^{(n)};{\bf w}) \tag{2}
$$

これは、情報理論における交差エントロピーと等価である。

## 確率的勾配降下法による負の対数尤度の最小化

### 連鎖律の適用

記載の簡単化のため、$z_n = \tilde{Y}({\bf x}^{(n)};{\bf w}) = softmax ({\bf w^T x})$、$E = -\ln P$ とすると、以下のように記載できる。

$$
E = -\ln P = -\sum^N_{n=1}\sum^k_{i=1} y_i^{(n)} \ln z_{in} \tag{3}
$$


確率的勾配降下法では、データの一部に対して勾配 ${\frac{\partial E}{\partial {\bf w}}}$ を計算し、学習率 $\eta$ で減衰させた値 $\eta \frac{\partial E}{\partial {\bf w}}$ を現在の重み${\bf w}$に対して引いていくことで、現在よりも良い解 ${\bf w}^{new}$ を得る。  
これを式で表すと以下の通り。

$$
{\bf w}^{new} \leftarrow {\bf w} - \eta \frac{\partial E}{\partial {\bf w}} \tag{4}
$$

*※注: 二値分類で対数尤度最大化を解いていた場合は勾配を上昇する方向に更新する必要があったため、勾配を足していたが、負の対数尤度最小化を解く場合は勾配を引く必要がある。*

クラス $h$ の確率を推定する重み ${\bf w_h}$ の、$m$ 番目の重み $w_{hm}$ を考えると、上式同様、

$$
w_{hm}^{new} \leftarrow w_{hm} - \eta \frac{\partial E}{\partial {w_{hm}}} \tag{5}
$$

と表すことができるため、$w_{hm}^{new}$ は、対数尤度の勾配 $\frac{\partial E}{\partial {w_{hm}}}$ が求まれば算出できる。ただし、$E$ は $z_{in}$ の関数であり、直接 $w_{hm}$ で微分することは難しいため、以下のように連鎖律（chain rule）を適用すると、下記にように表すことができる。

$$
\frac{\partial E}{\partial {w_{hm}}} = 
\frac{\partial E}{\partial z_{in}} \frac{\partial z_{in}}{\partial w_{hm}} \tag{6}
$$

### $\frac{\partial E}{\partial z_{in}}$ の導出

対数関数の微分: $(\log x)' = \frac{1}{x}$ を用いると、$\ln P$ の $z_{in}$ に関する微分は式(2) を用いて下記のように表すことができる。

$$
\frac{\partial E}{\partial z_{in}} = 
-\sum_{n=1}^N\sum_{i=1}^k \frac{y_i^{(n)}}{z_{in}} \tag{7}
$$

### $\frac{\partial z_{in}}{\partial w_{hm}}$ の導出

ソフトマックス関数の微分は下記の通り

$$
\frac{\partial y_i}{\partial x_j} = 
\begin{cases}
    y_i(1-y_i) & i = j\\
    -y_iy_j & i \neq j
\end{cases}
$$

これを用いると、$z_{in} = sigmoid ({\bf w^T x})$ の $w_{hm}$ に関する偏微分は下記のように表すことができる。

$$
\frac{\partial z_{in}}{\partial w_{hm}} =
\begin{cases} 
   z_h(1-z_h)[x_n]_m & i = h\\
    -z_iz_h[x_n]_m & i \neq h
\end{cases}
$$

ただし、$[x_n]_m$ は、n個目のデータのm番目の説明変数である。（0番目の説明変数はバイアス項: 1 とする）

### $\frac{\partial E}{\partial w_{hm}}$ の導出から $w_{hm}^{new}$ の導出まで

式(6), (7) から、$\ln P$ の $w_m$ に関する勾配は、下記のように求まる。

$$
\begin{align} 
\frac{\partial E}{\partial {w_{hm}}} 
&= \frac{\partial E}{\partial z_{in}} \frac{\partial z_{in}}{\partial w_{hm}} \\
&= -\sum_{n=1}^N  \Bigl\{ \frac{y_h^{(n)}}{z_{hn}} z_{hn} (1-z_{hn}) - \sum_{i\neq h} \frac{y_i^{(n)}}{z_{in}}z_{in}z_{hn} \Bigl\} [x_n]_m \\
&= -\sum_{n=1}^N  \Bigl\{ y_h^{(n)}(1 -z_{hn}) - \sum_{i\neq h} y_i^{(n)}z_{hn} \Bigl\} [x_n]_m \\
&= -\sum_{n=1}^N \Bigl\{ y_h^{(n)} - \sum_{i=1}^k  y_i^{(n)}z_{hn} \Bigl\} [x_n]_m \\
&= \sum_{n=1}^N \Bigl\{ z_{hn} -  y_h^{(n)} \Bigl\} [x_n]_m \\
\end{align}
$$

従って、$w_{hm}^{new}$ は下記の通り。

$$
w_{hm}^{new} = w_{hm} + \eta w_{hm}^{new} = w_{hm} + \eta \sum_{n=1}^N \Bigl\{ z_{hn} -  y_h^{(n)} \Bigl\} [x_n]_m 
$$